In [9]:
import pypsa
def fix_artificial_lines_reasonable(network):
    """
    Fix artificial lines with reasonable capacity values:
    - s_nom = based on connected bus demand (with safety factor)
    - s_nom_extendable = False (non-extendable)
    - Keep capacity high enough to meet demand
    """
    print("=== FIXING ARTIFICIAL LINES WITH REASONABLE CAPACITY ===")

    # Find artificial lines
    artificial_lines = [line for line in network.lines.index
                       if any(keyword in str(line).lower() for keyword in ['new', '<->', 'artificial'])]

    if not artificial_lines:
        # If no artificial lines found by name, look for lines with s_nom=0
        # which is often a sign of artificial lines
        zero_capacity_lines = network.lines[network.lines.s_nom == 0].index.tolist()
        if zero_capacity_lines:
            artificial_lines = zero_capacity_lines

    print(f"Found {len(artificial_lines)} artificial lines to fix:")

    # Get maximum demand per bus across all snapshots
    bus_max_demand = {}
    for bus in network.buses.index:
        bus_demand = 0
        for load_name, load in network.loads.iterrows():
            if load.bus == bus and load_name in network.loads_t.p_set.columns:
                bus_demand = max(bus_demand, network.loads_t.p_set[load_name].max())
        bus_max_demand[bus] = bus_demand

    # Fix each artificial line with reasonable capacity
    for line_name in artificial_lines:
        # Get connected buses
        bus0 = network.lines.loc[line_name, 'bus0']
        bus1 = network.lines.loc[line_name, 'bus1']

        # Get maximum demand at these buses
        bus0_demand = bus_max_demand.get(bus0, 0)
        bus1_demand = bus_max_demand.get(bus1, 0)

        # Calculate required capacity with safety factor
        # Use 3x the higher demand to ensure adequate capacity
        safety_factor = 3.0
        required_capacity = max(bus0_demand, bus1_demand) * safety_factor

        # Ensure minimum reasonable capacity (1000 MW)
        required_capacity = max(required_capacity, 1000)

        print(f"\n Fixing: {line_name}")
        print(f"    Connected buses: {bus0} ↔ {bus1}")
        print(f"    Bus demands: {bus0}: {bus0_demand:.1f} MW, {bus1}: {bus1_demand:.1f} MW")

        # Set s_nom to required capacity
        old_s_nom = network.lines.loc[line_name, 's_nom']
        network.lines.loc[line_name, 's_nom'] = required_capacity
        print(f"    s_nom: {old_s_nom} → {required_capacity:.1f} MW")

        # Make sure line is not extendable
        if 's_nom_extendable' not in network.lines.columns:
            network.lines['s_nom_extendable'] = False
        network.lines.loc[line_name, 's_nom_extendable'] = False
        print(f"    s_nom_extendable: → False")

    return network

def create_pypsa_network(network_file):
    """Create a PyPSA network from the .nc file."""
    # Initialize network
    network = pypsa.Network(network_file)
    for storage_name in network.storage_units.index:
        # Use .loc for direct assignment to avoid SettingWithCopyWarning
        network.storage_units.loc[storage_name, 'cyclic_state_of_charge'] = False

        # Set marginal_cost to 0.01
        network.storage_units.loc[storage_name, 'marginal_cost'] = 0.01

        # Set marginal_cost_storage to 0.01
        network.storage_units.loc[storage_name, 'marginal_cost_storage'] = 0.01

        # Set spill_cost to 0.1
        network.storage_units.loc[storage_name, 'spill_cost'] = 0.1

        network.storage_units.loc[storage_name, 'efficiency_store'] = 0.866025 #use phs efficiency (hydro didnt have an efficiency, but i want to model them all as the same)

        # Fix unrealistic max_hours values
        current_max_hours = network.storage_units.loc[storage_name, 'max_hours']

        if 'PHS' in storage_name:
            # PHS with missing data - set to typical range
            network.storage_units.loc[storage_name, 'max_hours'] = 8.0
            print(f"Fixed {storage_name}: set max_hours to 8.0")

        elif 'hydro' in storage_name:
            # Hydro with unrealistic data - set to validated range
            network.storage_units.loc[storage_name, 'max_hours'] = 6.0
            print(f"Fixed {storage_name}: corrected max_hours from {current_max_hours} to 6.0")


    fix_artificial_lines_reasonable(network)

    return network

In [12]:
network_file_path = "/Users/antoniagrindrod/Documents/pypsa-earth_project/pypsa-earth-RL/RL/elec_s_10_ec_lc1.0_1h.nc"
network=create_pypsa_network(network_file_path)

# Get slack generators first
slack_generators = network.generators[network.generators.control == "Slack"].index

# Check if any SLACK generator has zero p_nom (nameplate capacity)
zero_p_nom_slack = (network.generators.loc[slack_generators, 'p_nom'] == 0).any()
print(f"Any SLACK generators with zero p_nom: {zero_p_nom_slack}")

# Check if any SLACK generator has zero p_max_pu at any time
if hasattr(network.generators_t, 'p_max_pu') and not network.generators_t.p_max_pu.empty:
    # Check only slack generators in the time-varying data
    slack_in_timeseries = [gen for gen in slack_generators if gen in network.generators_t.p_max_pu.columns]
    if slack_in_timeseries:
        zero_p_max_pu_slack = (network.generators_t.p_max_pu[slack_in_timeseries] == 0).any().any()
        print(f"Any SLACK generators with zero p_max_pu at any time: {zero_p_max_pu_slack}")
    else:
        print("No slack generators found in time-varying p_max_pu data")
else:
    # p_max_pu is static in generators table - check only slack generators
    zero_p_max_pu_slack = (network.generators.loc[slack_generators, 'p_max_pu'] == 0).any()
    print(f"Any SLACK generators with zero p_max_pu: {zero_p_max_pu_slack}")

# Additional diagnostic: show the actual values for slack generators
print(f"\nSlack generator details:")
slack_details = network.generators.loc[slack_generators, ['p_nom', 'p_max_pu', 'p_min_pu']]
print(slack_details)

# Check the effective p_max for slack generators
slack_p_max_effective = slack_details['p_max_pu'] * slack_details['p_nom']
print(f"\nSlack generators with zero effective p_max: {(slack_p_max_effective == 0).any()}")
if (slack_p_max_effective == 0).any():
    zero_effective = slack_details[slack_p_max_effective == 0]
    print("Slack generators with zero effective p_max:")
    print(zero_effective)


INFO:pypsa.io:Imported network elec_s_10_ec_lc1.0_1h.nc has buses, carriers, generators, global_constraints, lines, loads, storage_units


Fixed ZA0 0 PHS: set max_hours to 8.0
Fixed ZA0 5 PHS: set max_hours to 8.0
Fixed ZA0 6 hydro: corrected max_hours from 3831.6270020496813 to 6.0
=== FIXING ARTIFICIAL LINES WITH REASONABLE CAPACITY ===
Found 3 artificial lines to fix:

 Fixing: lines new ZA0 4 <-> ZA2 0 AC
    Connected buses: ZA0 4 ↔ ZA2 0
    Bus demands: ZA0 4: 15945.8 MW, ZA2 0: 452.6 MW
    s_nom: 0.0 → 47837.3 MW
    s_nom_extendable: → False

 Fixing: lines new ZA0 0 <-> ZA1 0 AC
    Connected buses: ZA0 0 ↔ ZA1 0
    Bus demands: ZA0 0: 3513.0 MW, ZA1 0: 1386.9 MW
    s_nom: 0.0 → 10538.9 MW
    s_nom_extendable: → False

 Fixing: lines new ZA0 0 <-> ZA3 0 AC
    Connected buses: ZA0 0 ↔ ZA3 0
    Bus demands: ZA0 0: 3513.0 MW, ZA3 0: 721.1 MW
    s_nom: 0.0 → 10538.9 MW
    s_nom_extendable: → False
Any SLACK generators with zero p_nom: True
Any SLACK generators with zero p_max_pu at any time: True

Slack generator details:
                        p_nom  p_max_pu  p_min_pu
Generator                           

In [13]:
# Check if any line has zero s_nom
zero_s_nom = (network.lines['s_nom'] == 0).any()
print(f"Any lines with zero s_nom: {zero_s_nom}")

Any lines with zero s_nom: False


In [14]:
# Check if any storage unit has zero p_nom
zero_p_nom_storage = (network.storage_units['p_nom'] == 0).any()
print(f"Any storage units with zero p_nom: {zero_p_nom_storage}")

# Check if any storage unit has zero max_hours
zero_max_hours = (network.storage_units['max_hours'] == 0).any()
print(f"Any storage units with zero max_hours: {zero_max_hours}")

# Check if any storage unit has zero energy capacity (p_nom * max_hours)
storage_energy_capacity = network.storage_units['p_nom'] * network.storage_units['max_hours']
zero_energy_capacity = (storage_energy_capacity == 0).any()
print(f"Any storage units with zero energy capacity: {zero_energy_capacity}")

Any storage units with zero p_nom: False
Any storage units with zero max_hours: False
Any storage units with zero energy capacity: False


Yay! So to avoid division by zero in constraint checks, only need to deal with the zero nominal capacity of off-shore wind generators. we just remove them from the network. this removes 2 of the slack generators, so we need to see if the problem is still feasible, by running pypsa.optimize(). It is still feasible so we go ahead and remove the off-wind generators (missing data) from the network in the create_pypsa_network() function.

In [11]:
def test_remove_offshore_wind(network):
    """Test if removing offshore wind generators breaks system feasibility"""
    
    # First, identify offshore wind generators
    offwind_gens = network.generators[
        network.generators.index.str.contains('offwind', case=False, na=False)
    ].index
    
    print(f"Found {len(offwind_gens)} offshore wind generators:")
    print(offwind_gens.tolist())
    
    # Check their properties
    offwind_data = network.generators.loc[offwind_gens, ['p_nom', 'control', 'carrier']]
    print("\nOffshore wind generator details:")
    print(offwind_data)
    
    try:
        # Remove offshore wind generators one by one
        print(f"\nRemoving {len(offwind_gens)} offshore wind generators...")
        for gen in offwind_gens:
            network.remove("Generator", gen)
        
        # Try to run optimization
        print("Running PyPSA optimization...")
        network.optimize(solver_name='gurobi')
        
        print("✅ Optimization succeeded without offshore wind!")
        print(f"Objective value: {network.objective}")
        
        # Check if any slack generators are needed
        slack_power = network.generators_t.p[
            network.generators[network.generators.control == "Slack"].index
        ].sum(axis=1)
        
        print(f"Total slack power range: {slack_power.min():.1f} to {slack_power.max():.1f} MW")
        
    except Exception as e:
        print(f"❌ Optimization failed without offshore wind: {e}")
        print("Offshore wind generators may be essential for system balance")

# Run the test
test_remove_offshore_wind(network)

Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA0 5', 'ZA0 6', 'ZA0 7',
       'ZA0 8', 'ZA0 9', 'ZA1 0', 'ZA2 0', 'ZA3 0'],
      dtype='object', name='Bus')
Index(['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6',
       '7', '8', '9', 'lines new ZA0 4 <-> ZA2 0 AC',
       'lines new ZA0 0 <-> ZA1 0 AC', 'lines new ZA0 0 <-> ZA3 0 AC'],
      dtype='object', name='Line')


Found 12 offshore wind generators:
['ZA0 1 offwind-ac', 'ZA0 1 offwind-dc', 'ZA0 5 offwind-ac', 'ZA0 5 offwind-dc', 'ZA0 7 offwind-ac', 'ZA0 7 offwind-dc', 'ZA0 8 offwind-ac', 'ZA0 8 offwind-dc', 'ZA1 0 offwind-ac', 'ZA1 0 offwind-dc', 'ZA3 0 offwind-ac', 'ZA3 0 offwind-dc']

Offshore wind generator details:
                  p_nom control     carrier
Generator                                  
ZA0 1 offwind-ac    0.0          offwind-ac
ZA0 1 offwind-dc    0.0          offwind-dc
ZA0 5 offwind-ac    0.0          offwind-ac
ZA0 5 offwind-dc    0.0          offwind-dc
ZA0 7 offwind-ac    0.0          offwind-ac
ZA0 7 offwind-dc    0.0          offwind-dc
ZA0 8 offwind-ac    0.0          offwind-ac
ZA0 8 offwind-dc    0.0          offwind-dc
ZA1 0 offwind-ac    0.0   Slack  offwind-ac
ZA1 0 offwind-dc    0.0          offwind-dc
ZA3 0 offwind-ac    0.0   Slack  offwind-ac
ZA3 0 offwind-dc    0.0          offwind-dc

Removing 12 offshore wind generators...
Running PyPSA optimization...


Index(['ZA0 0', 'ZA0 1', 'ZA0 2', 'ZA0 3', 'ZA0 4', 'ZA0 5', 'ZA0 6', 'ZA0 7',
       'ZA0 8', 'ZA0 9', 'ZA1 0', 'ZA2 0', 'ZA3 0'],
      dtype='object', name='Bus')
Index(['0', '1', '10', '11', '12', '13', '14', '15', '2', '3', '4', '5', '6',
       '7', '8', '9', 'lines new ZA0 4 <-> ZA2 0 AC',
       'lines new ZA0 0 <-> ZA1 0 AC', 'lines new ZA0 0 <-> ZA3 0 AC'],
      dtype='object', name='Line')
INFO:linopy.model: Solve problem using Gurobi solver
INFO:linopy.io:Writing objective.
Writing continuous variables.: 100%|██████████| 6/6 [00:01<00:00,  5.15it/s]
INFO:linopy.io: Writing time: 8.16s


Set parameter Username


INFO:gurobipy:Set parameter Username


...

Solved in 43505 iterations and 15.24 seconds (5.51 work units)


INFO:gurobipy:Solved in 43505 iterations and 15.24 seconds (5.51 work units)


Optimal objective  9.808257026e+09


INFO:gurobipy:Optimal objective  9.808257026e+09
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 630720 primals, 1445400 duals
Objective: 9.81e+09
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Line-fix-s-lower, Line-fix-s-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, Kirchhoff-Voltage-Law, StorageUnit-energy_balance were not assigned to the network.


✅ Optimization succeeded without offshore wind!
Objective value: 9808257026.437061
Total slack power range: 0.0 to 5743.0 MW
